### DECISION TREES PART I
Jakub Prusak 151178
<br> Katarzyna Przepiórska 151752

#### 1. Wstęp
Zadanie polega na stworzeniu drzewa decyzyjnego na podstawie danych zawierających informacje o podróżujących Tytanikiem.

#### 2. Wczytanie danych z pliku
Pierwszym krokiem jest wczytanie danych z pliku .csv i zapisanie ich jako wartości w słowniku ``passengers``.

In [1]:
import csv
import math

#PassengerId, Pclass - class on ship, Name, Sex, Age, 
#SibSp - sibling they were traveled with, Parch - parents and children, Survived - did they survive

passengers = {
    "id" : [],
    "p_class" : [],
    "name" : [],
    "sex" : [],
    "age" : [],
    "siblings" : [],
    "children" : [],
    "survived" : []
}

with open("titanic-homework.csv", "r") as file:
    titanicFile = csv.DictReader(file)
    titanicList = list(titanicFile)
    print(titanicList)
    print(2)
    for line in titanicList:
        passengers["id"].append(line['PassengerId'])
        passengers["p_class"].append(line['Pclass'])
        passengers["name"].append(line['Name'])
        passengers["sex"].append(line['Sex'])
        passengers["age"].append(line['Age'])
        passengers["siblings"].append(line['SibSp'])
        passengers["children"].append(line['Parch'])
        passengers["survived"].append(line['Survived'])

print (passengers)

[{'PassengerId': '1', 'Pclass': '3', 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': '22', 'SibSp': '1', 'Parch': '0', 'Survived': '0'}, {'PassengerId': '2', 'Pclass': '1', 'Name': 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Sex': 'female', 'Age': '38', 'SibSp': '1', 'Parch': '0', 'Survived': '1'}, {'PassengerId': '3', 'Pclass': '3', 'Name': 'Heikkinen, Miss. Laina', 'Sex': 'female', 'Age': '26', 'SibSp': '0', 'Parch': '0', 'Survived': '1'}, {'PassengerId': '4', 'Pclass': '1', 'Name': 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'Sex': 'female', 'Age': '35', 'SibSp': '1', 'Parch': '0', 'Survived': '1'}, {'PassengerId': '5', 'Pclass': '3', 'Name': 'Allen, Mr. William Henry', 'Sex': 'male', 'Age': '35', 'SibSp': '0', 'Parch': '0', 'Survived': '0'}, {'PassengerId': '6', 'Pclass': '3', 'Name': 'Moran, Mr. James', 'Sex': 'male', 'Age': '34', 'SibSp': '0', 'Parch': '0', 'Survived': '0'}, {'PassengerId': '7', 'Pclass': '1', 'Name': 'McCarthy, Mr. Timothy J', 'Sex': 'ma

#### 3. Obliczanie prawdopodobieństwa
Przed przystąpieniem do następnego kroku, utworzona została funkcja ``get_probability()`` obliczająca prawdopodobieństwo przeżycia lub nieprzeżycia dla każdej wartości klasy. Funkcja zwraca słownik, w którym kluczami są nazwy klas, a wartościami listy. Pola w każdej z takich list oznaczają kolejno: prawdopodobieństwo przeżycia, prawdopodobieństwo nieprzeżycia oraz stosunek liczności danej klasy do wszystkich pasażerów (wartość ta będzie wykorzystana w obliczaniu entropii warunkowej).

In [148]:
def get_probability(passengers,key):
    new_dict  = {}
    values = []
    i = 0
    
    if key == 'age':
        new_dict  = {"young":[0,0],"middle":[0,0],"old":[0,0]}
        for age in passengers[key]:
            if(int(age)<=20):
                x='young'
            elif(int(age)<=40):
                x='middle'
            else:
                x='old'
            if(passengers["survived"][i] == '1'):
                new_dict[x][1] += 1
            else:
                new_dict[x][0] += 1
            i+=1
    else:    
        for x in passengers[key]:
            if x not in values:
                values.append(x)
                new_dict[x] = [0,0]
            if(passengers["survived"][i] == '1'):
                new_dict[x][1] += 1
            else:
                new_dict[x][0] += 1
            i+=1
        
     
    probability_dict = {}  
    for field in new_dict:
        probability_dict[field]=[new_dict[field][0]/(new_dict[field][0] + new_dict[field][1]),new_dict[field][1]/(new_dict[field][0] + new_dict[field][1])]
        probability_dict[field].append((new_dict[field][0] + new_dict[field][1])/i)
    return probability_dict

# probabilities = {}
# probabilities['p_class'] = get_probability(passengers, 'p_class')
# probabilities['sex'] = get_probability(passengers, 'sex')
# probabilities['siblings'] = get_probability(passengers, 'siblings')
# probabilities['children'] = get_probability(passengers, 'children')
# probabilities['age'] = get_probability(passengers, 'age')

#### 4. Entropia, entropia warunkowa i information gain
Aby podjąć decyzję dotyczącą wyboru następnego węzła, wybrać należy ten atrybut, który ma największą wartość **information gain**, czyli różnicy **entropii** i **entropii warunkowej**.
W tym celu utworzono dwie funkcje - ``entropy()`` i ``conditional_entropy()`` wykorzystywane później do obliczenia wartości information gain, która zapisywana jest w słowniku o tej samej nazwie. W funkcji ``choose_leaf()`` wybierany jest następny liść drzewa na podstawie największej wartości innformation gain. Wykorzystano przedstawione na zajciach wzory:
$$
Entropy(S) = -\sum_{i=1}^n P(x_{i})logP(x_{i})
$$
$$
Entropy(S|A) = \sum_{j=1}^m \frac{|S_{j}|}{|S|}Entropy(S_{j})
$$
$$
Gain(S,A) = Entropy(S) − Entropy(S|A)
$$

In [185]:
probabilities = {}
probabilities['p_class'] = get_probability(passengers, 'p_class')
probabilities['sex'] = get_probability(passengers, 'sex')
probabilities['siblings'] = get_probability(passengers, 'siblings')
probabilities['children'] = get_probability(passengers, 'children')
probabilities['age'] = get_probability(passengers, 'age')

def entropy(lst):
    p = 0
    for x in lst:
        if x:
           p -= x * math.log2(x)
    return p

def conditional_entropy(probabilities, key):
    e_sum = 0
    for un_key in probabilities[key]:
        e_sum += probabilities[key][un_key][-1] * entropy(probabilities[key][un_key][:-1])
    return e_sum


def choose_leaf(probabilities):
    information_gain = {}
    max_ratio = 0
    max_ratio_key = ""
    for key in probabilities:
        information_gain[key] = 1 - conditional_entropy(probabilities, key)
        if information_gain[key] > max_ratio:
            max_ratio = information_gain[key]
            max_ratio_key = key
    return max_ratio, max_ratio_key, information_gain

choose_leaf(probabilities)
        

(0.4205719940747936,
 'sex',
 {'p_class': 0.11072176334439654,
  'sex': 0.4205719940747936,
  'siblings': 0.06970485312408614,
  'children': 0.04563068395017167,
  'age': 0.03838565357775392})

#### 5. Intrinsic information i gain ratio
W przypadku dużej ilości różych wartości można wykorzystać wartość **gain ratio**, czyli wartość information gain znormalzowaną z użyciem **intrinsic information**, które pod uwagę bierze wielkość poszczególnych gałęzi. Gain ratio rozpatruje nie tylko czystość węzłów, ale także rozłożenie przykładów poniędzy gałęzie drzewa. W celu obliczenia obu wartości zaimplementowano funkcje ``intrinsic_info()`` oraz ``gain_ratio()``. Do wykonania obu obliczeń wykorzystane zostały następujące wzory:
$$
IntrinsicInfo(S,A) = -\sum_{i=1}^n \frac{|S_{j}|}{|S|}log_{2}\frac{|S_{j}|}{|S|}
$$
$$
GainRatio(S,A) = \frac{Gain(S,A)}{IntrinsicInfo(S,A)}
$$

In [215]:
def intrinsic_info(probabilities, key):
    e_sum = 0
    for un_key in probabilities[key]:
        e_sum -= probabilities[key][un_key][-1] * math.log2(probabilities[key][un_key][-1])
    return e_sum

def gain_ratio(probabilities):
    gain_ratio ={}
    for key in probabilities:
        gain_ratio[key] = choose_leaf(probabilities)[2][key]/(intrinsic_info(probabilities, key)) 
    return gain_ratio

gain_ratio(probabilities)


{'p_class': 0.08080530167007237,
 'sex': 0.4331548860228121,
 'siblings': 0.04305209220247883,
 'children': 0.040288443395889766,
 'age': 0.02574962077508204}

#### 6. Utworzenie gałęzi drzewa i wyświetlanie drzewa

In [1]:
def split_by_key(passengers,key):
    values = []
    res = {key:{}}
    i = 0    
    if key == 'age':
        res[key]  = {"young":{},"middle":{},"old":{}}
        for age in passengers[key]:
            if(int(age)<=20):
                x='young'
            elif(int(age)<=40):
                x='middle'
            else:
                x='old'
            for field in passengers:
                if field not in res[key][x]:
                    res[key][x][field]=[]
                res[key][x][field].append(passengers[field][i])
            i+=1
    else:    
        for k in passengers[key]:
            if k not in values:
                values.append(k)
                res[key][k]={}
            for field in passengers:
                if field not in res[key][k]:
                    res[key][k][field]=[]
                res[key][k][field].append(passengers[field][i])
            i+=1
            
    return res
        
def print_tree(root:dict,cd,md):
    # if(cd>=md):
    #     return  
    if("survived" in root):
        if((root["survived"].count("1") == len(root["survived"])) or (root["survived"].count("0") == len(root["survived"]))):
            print(' '*(cd-1) +chr(9492)+ "survived")
            if(root["survived"][0]=="1"):
                print(' '*cd +chr(9492)+ "yes")
                return
            print(' '*cd +chr(9492)+ "no")
            
    for key in root:
        if(key=="id"):
            return
        if(cd != 0):
            print(' '*(cd-1) +chr(9492)+ key)
        else:
            print(key)
        print_tree(root[key],cd+1,md)
        
def build_tree(root,cd=0,md=2):
    if(cd>=md):
        return({id:0})
    print(cd)
    probabilities = {}
    probabilities['p_class'] = get_probability(root, 'p_class')
    probabilities['sex'] = get_probability(root, 'sex')
    probabilities['siblings'] = get_probability(root, 'siblings')
    probabilities['children'] = get_probability(root, 'children')
    probabilities['age'] = get_probability(root, 'age')
    print(probabilities)
    tree = {}
    
    # information_gain 
    _,best_key,_ = choose_leaf(probabilities)
    
    tree = split_by_key(root,best_key)
    
    for key in tree[best_key]:
        print("key",key)
        tree[best_key][key]=build_tree(tree[best_key][key],cd+1)
        # probabilities['p_class'] = get_probability(tree[best_key][key], 'p_class')
        # probabilities['sex'] = get_probability(tree[best_key][key], 'sex')
        # probabilities['siblings'] = get_probability(tree[best_key][key], 'siblings')
        # probabilities['children'] = get_probability(tree[best_key][key], 'children')
        # probabilities['age'] = get_probability(tree[best_key][key], 'age')
        # _,d_best_key,_ = choose_leaf(probabilities)
        # tree[best_key][key]=split_by_key(tree[best_key][key],d_best_key)
    
    # done = False
    # i = 1
    
    # while(not done):
    
    return tree
            
            
    
    
# def choose_leaf(probabilities):
#     information_gain = {}
#     max_ratio = 0
#     max_ratio_key = ""
#     for key in probabilities:
#         information_gain[key] = 1 - conditional_entropy(probabilities, key)
#         if information_gain[key] > max_ratio:
#             max_ratio = information_gain[key]
#             max_ratio_key = key
#     return max_ratio, max_ratio_key, information_gain
    
# def entropy(lst):
#     p = 0
#     for x in lst:
#         if x:
#            p -= x * math.log2(x)
#     return p

# def conditional_entropy(probabilities, key):
#     e_sum = 0
#     for un_key in probabilities[key]:
#         e_sum += probabilities[key][un_key][-1] * entropy(probabilities[key][un_key][:-1])
#     return e_sum

        
# tree = split_by_key(passengers,"p_class")
# tree["p_class"]["2"] = split_by_key(tree["p_class"]["2"],"sex")

tree = build_tree(passengers)

print_tree(tree,0,3)


NameError: name 'passengers' is not defined